In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tempfile, sys, os
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  


sys.path.insert(0, os.path.abspath('..'))
import keras.backend as K

from deepexplain.tensorflow import DeepExplain
from keras.layers import Dense, Activation
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
import numpy as np

image_instance = np.array([np.random.random((224, 224, 3))])


# Load VGG16 model and change last layer to be split with dense and Activation
vgg = VGG16(include_top=True, weights='imagenet')
vgg.save_weights('imagenet.h5')
x = Dense(1000)(vgg.layers[-2].output)
x = Activation('softmax')(x)
vgg_new = Model(vgg.input, x)
vgg_new.load_weights('imagenet.h5')

Using TensorFlow backend.


In [2]:
# Get activations of layer [-7]
conv_act_model = Model(inputs=vgg_new.input, outputs=vgg_new.layers[-7].output)
conv_act = conv_act_model.predict(image_instance)

In [3]:
# Get contributions of convolutional layer [-7]
with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
    input_tensor = vgg_new.layers[-7].input
    fModel = Model(inputs=input_tensor, outputs = vgg_new.layers[-2].output)
    target_tensor = fModel(input_tensor)

    xs = conv_act
    ys = np.zeros((1000))
    ys[900] = 1

    answer = de.explain('elrp', target_tensor * ys, input_tensor, xs)

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:180: UserWarning: Model inputs must come from `keras.layers.Input` (thus holding past layer metadata), they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to your model was not an Input tensor, it was generated by layer block5_conv2.
Note that input tensors are instantiated via `tensor = keras.layers.Input(shape)`.
The tensor that caused the issue was: block5_conv2_2/Relu:0
  str(x.name))


ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_3:0", shape=(?, 224, 224, 3), dtype=float32) at layer "input_3". The following previous layers were accessed without issue: []